In [ ]:
#importing all libs

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
# Try reading the CSV with a different encoding
df=pd.read_csv('spam.csv', encoding='latin-1')

In [ ]:
df.sample()

In [ ]:
df.shape

##Data cleaning


In [ ]:
df.info()

In [ ]:
df.drop(columns=["Unnamed: 2","Unnamed: 3","Unnamed: 4"],inplace=True)

In [ ]:
df.head(2)

In [ ]:
df.rename(columns={"v1":"target","v2":"text"},inplace=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()

In [ ]:
encoder.fit_transform(df["target"])

In [ ]:
# Apply the transformation to the 'target' column
df["target"] = encoder.fit_transform(df["target"])

missing values

In [ ]:
df.isnull().sum()

In [ ]:
#check duplicate values
df.duplicated().sum()

In [ ]:
#remove duplicats
df=df.drop_duplicates(keep="first")

In [ ]:
df.shape

## 2.EDA

In [ ]:
df["target"].value_counts()

In [ ]:
import matplotlib.pyplot as plt
plt.pie(df["target"].value_counts(),labels=["ham","spam"],autopct="%0.2f")

## 3.Data preprocessing

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')


In [ ]:
from nltk.corpus import stopwords
import string
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()


In [ ]:
def transform_text(text):
  text=text.lower()
  text=nltk.word_tokenize(text)
  y=[]
  for i in text :
      if i.isalnum():
        y.append(i)
  text=y[:]
  y.clear()
  for i in text:
    if i not in stopwords.words('english') and i not in string.punctuation:
       y.append(i)

  text=y[:]
  y.clear()

  for i in text:
    y.append(ps.stem(i))
  return y

In [ ]:
transform_text("I'm gonna be home soon and i don't want to talk about this stuff anymore tonight, k? I've cried enough today.")

In [ ]:
df["transformed_text"]=df["text"].apply(transform_text)

In [ ]:
df.head()

In [ ]:
#word cloud of spam sms
!pip install wordcloud


In [ ]:
#word cloud of spam sms
!pip install wordcloud
from wordcloud import WordCloud
wc=WordCloud(width=500,height=500,min_font_size=10,background_color="white")

In [ ]:
spam_wc=wc.generate(df[df["target"]==0]["transformed_text"].str.cat(sep=" "))


In [ ]:
# word cloud of spam sms
!pip install wordcloud
from wordcloud import WordCloud
wc=WordCloud(width=500,height=500,min_font_size=10,background_color="white")

# Join the list of words into a single string for each row
df["transformed_text_str"] = df["transformed_text"].apply(lambda x: " ".join(x))

# Generate the word cloud using the new string column
spam_wc=wc.generate(df[df["target"]==1]["transformed_text_str"].str.cat(sep=" "))

In [ ]:
plt.figure(figsize=(5,5))
plt.imshow(spam_wc)

In [ ]:
ham_wc=wc.generate(df[df["target"]==0]["transformed_text_str"].str.cat(sep=" "))

In [ ]:
plt.figure(figsize=(5,5))
plt.imshow(ham_wc)

In [ ]:
#top 30 words
spam_corpus=[]
for msg in df[df["target"]==1]["transformed_text_str"].tolist():
    for word in msg.split():
        spam_corpus.append(word)

In [ ]:
len(spam_corpus )

In [ ]:
spam_corpus[:20]

In [ ]:
#top 30 words
spam_corpus=[]
for msg in df[df["target"]==0]["transformed_text_str"].tolist():
    for word in msg.split():
        spam_corpus.append(word)

In [ ]:
spam_corpus[:20]


## 4.Model Building

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
cv=CountVectorizer()
tdf=TfidfVectorizer(max_features=3000)

In [ ]:
x=tdf.fit_transform(df["transformed_text_str"]).toarray()

In [ ]:
x.shape

In [ ]:
y=df["target"].values

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

In [ ]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score
gnb=GaussianNB()
mnb=MultinomialNB()
bnb=BernoulliNB()

In [ ]:
gnb.fit (x_train,y_train)
y_pred1=gnb.predict(x_test)
print(accuracy_score(y_test,y_pred1))

In [ ]:
mnb.fit (x_train,y_train)
y_pred2=mnb.predict(x_test)
print(accuracy_score(y_test,y_pred2))

In [ ]:
bnb.fit (x_train,y_train)
y_pred3=bnb.predict(x_test)
print(accuracy_score(y_test,y_pred3))

In [ ]:

lrc = LogisticRegression(solver='liblinear', penalty='l1')
nv=BernoulliNB(alpha=0.5,binarize=None)
svc = SVC(kernel='sigmoid', gamma=1.0)
dtc = DecisionTreeClassifier(max_depth=5)
rfc = RandomForestClassifier(n_estimators=50, random_state=2)
knc = KNeighborsClassifier(n_neighbors=5)

clfs = {
    'SVC' : svc,
    'NV': nv,
    'LRC': lrc,
    'KN' : knc,
    'DT': dtc,
    'RF': rfc,
}


In [ ]:
# Function to train and evaluate a classifier
def train_classifier(clf, x_train, y_train, x_test, y_test):
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    return accuracy, precision

In [ ]:
train_classifier(svc, x_train, y_train, x_test, y_test)

In [ ]:
accuracy_scores.append(current_accuracy)
precision_scores.append(current_precision)

In [ ]:

# Train and evaluate each classifier
accuracy_scores = []
precision_scores = []
algorithm_names = []

for name, clf in clfs.items():
    current_accuracy, current_precision = train_classifier(clf, x_train, y_train, x_test, y_test)
    accuracy_scores.append(current_accuracy)
    precision_scores.append(current_precision)
    algorithm_names.append(name)

    print("for",name)
    print("Accuracy:", current_accuracy)
    print("Precision:", current_precision)



performance_df = pd.DataFrame({'Algorithm': algorithm_names, 'Accuracy': accuracy_scores, 'Precision': precision_scores}).sort_values('Precision', ascending=False)


In [ ]:
performance_df_acc = pd.DataFrame({'Algorithm': algorithm_names, 'Accuracy': accuracy_scores, 'Precision': precision_scores}).sort_values('Accuracy', ascending=False)


In [ ]:
performance_df_acc

In [ ]:
performance_df